In [19]:
# Cài đặt Poppler và các thư viện cần thiết
!apt-get update -qq
!apt-get install -qq -y poppler-utils          # poppler-utils để pdf2image hoạt động
!pip install --quiet pdf2image google-generativeai python-docx
!pip install pypdf
!pip install pymupdf


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [21]:
from google.colab import userdata

In [29]:
import os
from pdf2image import convert_from_path
from io import BytesIO
from PIL import Image
from docx import Document
import google.generativeai as genai

# Bước này giả sử bạn đã có API Key của Gemini 1.5 Pro gán vào biến môi trường.
# Ví dụ: bạn có thể gán trực tiếp như dưới (hoặc set môi trường trên giao diện Colab).
# os.environ["GENAI_API_KEY"] = "PUT_YOUR_API_KEY_HERE"

# Lấy API Key từ biến môi trường (hoặc gán trực tiếp nếu muốn)
GEMINI_API_KEY = userdata.get('gemini-2')

# Cấu hình cho SDK của Gemini
genai.configure(api_key=GEMINI_API_KEY)
# Khởi tạo model Gemini 1.5 Pro
model = genai.GenerativeModel(model_name="gemini-2.0-flash")


In [30]:
import fitz  # PyMuPDF
from PIL import Image
from io import BytesIO
import google.generativeai as genai
from docx import Document

# Cấu hình
GEMINI_API_KEY = userdata.get('gemini-2')
PDF_FILE = "/content/your_pdf_here.pdf"
OUTPUT_FILE = "extracted_text.docx"

# Setup Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Xử lý PDF
doc = fitz.open(PDF_FILE)
output_doc = Document()

print(f"Đang xử lý {len(doc)} trang...")

for page_num in range(len(doc)):
    page = doc[page_num]

    # Convert trang thành ảnh
    pix = page.get_pixmap(dpi=300)
    img_bytes = BytesIO(pix.tobytes("png"))
    image = Image.open(img_bytes)

    prompt_message = (
    "Extract all visible text from the image and return it in Markdown format. "
    "Ensure that the extracted content is accurately represented without additional descriptions, comments, or formatting. "
    "Do not invent any text or symbols; only return the content that is visibly present in the image."
    "No need to add page number infomation, often appear at the bottom"
  )

    # OCR với Gemini
    response = model.generate_content([
    prompt_message,
    image
    ])

    # Lưu kết quả
    if response.text:
        output_doc.add_heading(f"Trang {page_num + 1}", level=2)
        output_doc.add_paragraph(response.text.strip())

    print(f"✓ {page_num + 1}/{len(doc)}")

# Lưu file
doc.close()
output_doc.save(OUTPUT_FILE)
print(f"Hoàn thành: {OUTPUT_FILE}")

Đang xử lý 6 trang...
✓ 1/6
✓ 2/6
✓ 3/6
✓ 4/6
✓ 5/6
✓ 6/6
Hoàn thành: extracted_text.docx
